# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## step2 加载数据集

In [2]:
dataset = load_dataset('csv', data_files="./ChnSentiCorp_htl_all.csv", split='train')
dataset = dataset.filter(lambda x: x['review'] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## step4 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

C:\Users\yj.wang\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yj.wang\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

## step5 创建模型

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")  # 这里不需要设置CUDA了，因为Trainer会自动做判断

pytorch_model.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## step6 创建评估函数

In [1]:
import evaluate


In [3]:
evaluate.load("seqeval")

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [ ]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [9]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

### 创建训练参数

In [ ]:
train_args = TrainingArguments(output_dir="./checkpoints", # 指定训练输出的路径
                               per_device_train_batch_size=1, # 训练时的batch_size
                               gradient_accumulation_steps=32,  # 梯度累加，累加到32 batch的时候才计算梯度
                               gradient_checkpointing=True,
                               optim="adafactor",
                               per_device_eval_batch_size=32,  # 验证时的batch_size
                               logging_steps=10,  # log打印频率
                               evaluation_strategy="epoch",
                               save_strategy="epoch",  # 按照epoch的方式去保存
                               save_total_limit=3,  # 最多保存3个
                               learning_rate=2e-5,
                               weight_decay=0.01,
                               metric_for_best_mdoel="f1",  # 设置评估指标
                               load_best_model_at_end=True)  # 训练完之后加载最优模型
train_args

### 创建Trainer

In [ ]:
from transformers import DataCollatorWithPadding

for name, param in model.bert.named_parameters():
    param.requires_grad = False
    
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets['train'],
                  eval_dataset=tokenized_datasets['test'],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

### 模型训练

In [ ]:
trainer.train()

### 模型评估

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
sen = "我觉得这家酒店的菜不错， 饭很好吃！"
id2_label = {0:"差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, max_length=128, padding="max_length", truncation=True, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果： {id2_label.get(pred.item())}")
